In [1]:
#https://github.com/Jcharis/Natural-Language-Processing-Tutorials/blob/master/Text%20Classification%20With%20Machine%20Learning,SpaCy,Sklearn(Sentiment%20Analysis)/Text%20Classification%20&%20Sentiment%20Analysis%20with%20SpaCy,Sklearn.ipynb

In [2]:
# Load EDA packages
import pandas as pd

In [4]:
df = pd.read_excel('/Users/Suwani/Desktop/Moodys Project/Cleaned data/To analyze/Labelled data .xlsx')

In [5]:
df.head()

,Headline,my_label
0,Thai Lion Air makes it’s maiden voyage to Sri ...,0
1,Report on Sri Lankan reforms submitted to Pres...,0
2,Allowing Youth to Tuk-Tuk or not Tuk-Tuk: Shou...,0
3,The Spicy Goodness of Sri Lanka!,0
4,Sri lanka halfway between a flawed democracy a...,-1


In [6]:
df['my_label_1'] = df.my_label.replace(-1,0) 

In [7]:
df.head()

,Headline,my_label,my_label_1
0,Thai Lion Air makes it’s maiden voyage to Sri ...,0,0
1,Report on Sri Lankan reforms submitted to Pres...,0,0
2,Allowing Youth to Tuk-Tuk or not Tuk-Tuk: Shou...,0,0
3,The Spicy Goodness of Sri Lanka!,0,0
4,Sri lanka halfway between a flawed democracy a...,-1,0


In [8]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en')

In [9]:
# Build a list of stopwords to use to filter
stopwords = list(STOP_WORDS)

In [10]:
# ML Packages
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score 
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

In [11]:
#Custom transformer using spaCy 
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]
    def fit(self, X, y=None, **fit_params):
        return self
    def get_params(self, deep=True):
        return {}

# Basic function to clean the text 
def clean_text(text):     
    return text.strip().lower()

In [12]:
# Use the punctuations of string module
import string
punctuations = string.punctuation

In [13]:
# Creating a Spacy Parser
from spacy.lang.en import English
parser = English()

In [14]:
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    return mytokens

In [15]:
# Vectorization
vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1)) 
classifier = LinearSVC()


In [16]:
# Using Tfidf
tfvectorizer = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [17]:
# Splitting Data Set
from sklearn.model_selection import train_test_split

In [18]:
# Features and Labels
X = df['Headline']
ylabels = df['my_label_1']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2, random_state=42)


In [20]:
# Create the  pipeline to clean, tokenize, vectorize, and classify 
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', vectorizer),
                 ('classifier', classifier)])


In [21]:
# Fit our data
pipe.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x1a1e5aa780>),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function spacy_tokenizer at 0x1a1e46bb70>,
                                 vocabulary=None)),
                ('classifier',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                    

In [22]:
# Predicting with a test dataset
sample_prediction = pipe.predict(X_test)

In [23]:
# Prediction Results
# 1 = Positive review
# 0 = Negative review
for (sample,pred) in zip(X_test,sample_prediction):
    print(sample,"Prediction=>",pred)


HNB announces landmark partnership with CyberSource Prediction=> 0
Lankan envoy highlights expat workers' issues Prediction=> 0
BEYOND DEVOLUTION Prediction=> 0
Activists decry Sri Lankan president's praise for Duterte's drugs war Prediction=> 1
Women in Times of Disaster: Gender Dimension of Disaster Management in Sri Lanka Prediction=> 1
IMF to resume Sri Lanka's loan discussion in February: Lagarde Prediction=> 1
ComBank opens branch in Embuldeniya Prediction=> 0
ASPI drops Below 6,000 as Debt Concerns Weigh... Prediction=> 0
National Program to develop 2000 new exporters Prediction=> 0
Diverse voices Prediction=> 0
[CITIZENS' Mail - (21-01-2019)] Prediction=> 0
Weightlifter PUSHPARAJ he pushed to be hero and now he is ‘Housed’ Prediction=> 0
Herbal goodness for your baby Prediction=> 0
Sajeewa de Silva Prediction=> 0
[CITIZENS' Mail - (02-01-2019)] Prediction=> 0
Medical equipment for Kenyan children’s home Prediction=> 0
Sri Lanka’s Tourism achieves USD 4.4 bn revenue target for 2

Defense College Sports Meet Prediction=> 0
Increasing exports can improve jobs, raise wages - WB / ilo report Prediction=> 1
New horizons in education Prediction=> 0
Spice sector eyes techno collaborations, fresh investments Prediction=> 0
Railway TUs write to Minister regarding railway museum Prediction=> 0
Japanese Government provides US$624,382 for demining in North Prediction=> 0
Batting coach Jon Lewis goes on leave Prediction=> 0
Writ petition filed challenging Ranil’s Parliamentary seat fixed for support Prediction=> 0
71st National Day celebrated in Sri Lanka Prediction=> 0
A Sports Ombudsman Call Center to be established Prediction=> 1
Lanka look for turnaround in final tour match Prediction=> 0
RPT-FOCUS-Undeterred by Sanofi's stumble, Takeda takes similar path with dengue shot Prediction=> 1
Wickremesinghe proposes formation of national government in Sri Lanka Prediction=> 0
UPDATE 2-Sri Lanka cbank cuts banks' liquidity ratio to spur loan growth Prediction=> 1
Legendary gol

In [24]:
# Accuracy
print("Accuracy: ",pipe.score(X_test,y_test))
print("Accuracy: ",pipe.score(X_test,sample_prediction))

Accuracy:  0.6650485436893204
Accuracy:  1.0


In [25]:
# Accuracy
print("Accuracy: ",pipe.score(X_train,y_train))

Accuracy:  0.9993928354584092


In [26]:
# Another random review
pipe.predict(["This was a great movie"])

array([0])

In [27]:
example = ["I do enjoy my job",
 "What a poor product!,I will have to get a new one",
 "I feel amazing!"]

In [28]:
pipe.predict(example)

array([0, 0, 1])

In [29]:
#### Using Tfid

In [30]:
# Create the  pipeline to clean, tokenize, vectorize, and classify 
pipe_tfid = Pipeline([("cleaner", predictors()),
                 ('vectorizer', tfvectorizer),
                 ('classifier', classifier)])


In [31]:
pipe_tfid.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x1a1e98e898>),
                ('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words...
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function spacy_tokenizer at 0x1a1e46bb70>,
                                 use_idf=True, vocabulary=None)),
                ('classifier',
                 LinearSVC(C=1.0, class_weigh

In [32]:
sample_prediction1 = pipe_tfid.predict(X_test)

In [33]:
for (sample,pred) in zip(X_test,sample_prediction1):
    print(sample,"Prediction=>", pred)

HNB announces landmark partnership with CyberSource Prediction=> 0
Lankan envoy highlights expat workers' issues Prediction=> 0
BEYOND DEVOLUTION Prediction=> 0
Activists decry Sri Lankan president's praise for Duterte's drugs war Prediction=> 1
Women in Times of Disaster: Gender Dimension of Disaster Management in Sri Lanka Prediction=> 0
IMF to resume Sri Lanka's loan discussion in February: Lagarde Prediction=> 1
ComBank opens branch in Embuldeniya Prediction=> 0
ASPI drops Below 6,000 as Debt Concerns Weigh... Prediction=> 0
National Program to develop 2000 new exporters Prediction=> 0
Diverse voices Prediction=> 1
[CITIZENS' Mail - (21-01-2019)] Prediction=> 0
Weightlifter PUSHPARAJ he pushed to be hero and now he is ‘Housed’ Prediction=> 0
Herbal goodness for your baby Prediction=> 0
Sajeewa de Silva Prediction=> 0
[CITIZENS' Mail - (02-01-2019)] Prediction=> 0
Medical equipment for Kenyan children’s home Prediction=> 0
Sri Lanka’s Tourism achieves USD 4.4 bn revenue target for 2

In [34]:
print("Accuracy: ",pipe_tfid.score(X_test,y_test))
print("Accuracy: ",pipe_tfid.score(X_test,sample_prediction1))

Accuracy:  0.6893203883495146
Accuracy:  1.0
